In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv("IMDb Movies India.csv", encoding='latin1')

df.head()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali


In [3]:
df.isna().sum()

Name           0
Year         528
Duration    8269
Genre       1877
Rating      7590
Votes       7589
Director     525
Actor 1     1617
Actor 2     2384
Actor 3     3144
dtype: int64

In [4]:
df = df.dropna(subset=['Rating'])


In [5]:


df['Year'] = df['Year'].astype(str)


df['Year'] = df['Year'].str.replace(r'[()]', '', regex=True)


df['Year'] = pd.to_numeric(df['Year'], errors='coerce')


df['Year'] = df['Year'].fillna(df['Year'].median())



In [6]:

df['Duration'] = df['Duration'].astype(str)


df['Duration'] = df['Duration'].str.replace(' min', '', regex=False)


df['Duration'] = pd.to_numeric(df['Duration'], errors='coerce')


df['Duration'] = df['Duration'].fillna(df['Duration'].median())

In [7]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')
encoded_genres = encoder.fit_transform(df[['Genre']])


In [8]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np


X = df[['Year', 'Duration', 'Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3']]
y = df['Rating']


X_num = X[['Year', 'Duration']].values


encoder_genre = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder_director = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder_actor1 = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder_actor2 = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder_actor3 = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_genre = encoder_genre.fit_transform(X[['Genre']])
X_director = encoder_director.fit_transform(X[['Director']])
X_actor1 = encoder_actor1.fit_transform(X[['Actor 1']])
X_actor2 = encoder_actor2.fit_transform(X[['Actor 2']])
X_actor3 = encoder_actor3.fit_transform(X[['Actor 3']])


X_final = np.hstack([X_num, X_genre, X_director, X_actor1, X_actor2, X_actor3])


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

model = RandomForestRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R2 Score:", r2_score(y_test, y_pred))
